In [1]:
import pandas as pd
import chardet
import json
import re
import os
import openpyxl

In [ ]:
#df.reindex(colums = ['precio', 'producto_id', 'sucursal id'])

In [333]:
#carga manual de dataset
#TAREA: automatizarlo
dataset1 = 'Datasets_relevamiento_precios/precios_semana_20200413.csv' #Csv precio_semana
dataset2 = 'Datasets_relevamiento_precios/precios_semana_20200503.json' #json precio_semana
dataset3 = 'Datasets_relevamiento_precios/precios_semana_20200518.txt'#txt precio_semana
#dataset4 = 'Datasets_relevamiento_precios/precios_semanas_20200419_20200426.xlsx' #xls precio_semana
dataset5 = 'Datasets_relevamiento_precios/producto.parquet' #parquet producto
dataset6 = 'Datasets_relevamiento_precios/sucursal.csv' #csv sucursal



In [334]:
#abro el archivo y veo el encoding - 
#TAREA: buscar si hay otra forma autom. de hacerlo
def detect_encoding(dataset):
    '''
    Esta función sirve para detectar los tipo de enconding para distintos formatos
    '''
    with open(dataset, 'rb') as f:
        enc = chardet.detect(f.read())
    return enc['encoding']         

In [ ]:
def abrir_dataset():
    '''
    Guarda en un dataset el contenido de un archivo dependiendo su terminación
    '''
    return 

In [197]:
#dataset precio semana 1
precio_semana1 = pd.read_csv(dataset1, delimiter=",", encoding=detect_encoding(dataset1))

In [258]:
#ver la estrucura del df
precio_semana1

,precio,producto_id,sucursal_id
0,29.90,0000000001663,2-1-014
1,29.90,0000000002288,2-1-032
2,39.90,0000000002288,2-1-096
3,499.99,0000000205870,9-1-686
4,519.99,0000000205870,9-2-248
...,...,...,...
472161,139.99,9569753142128,25-1-1
472162,34.99,9795403001143,25-1-1
472163,312.50,9990385651922,5-1-3
472164,312.50,9990385651939,5-1-3


In [252]:
def busca_nulos(dataframe):
    return dataframe.isnull().sum()


In [253]:
#se buscan valores nulos en dataframe 1
busca_nulos(precio_semana1)

precio         13
producto_id    15
sucursal_id    15
dtype: int64

In [ ]:
# el % de nulos es relativamente bajo. Se mantienen para un futuro posible tratamiento en la étapa de análisis 
# y teniendo en cuenta el objetivo del negocio

In [199]:
#el precio es de tipo flotante por lo que no es necesaria ninguna modificacion
#precio_semana1['precio']=precio_semana1['precio'].astype(float)

In [275]:
#no parece haber nada raro en esto
precio_semana1['producto_id']

0         0000000001663
1         0000000002288
2         0000000002288
3         0000000205870
4         0000000205870
              ...      
472161    9569753142128
472162    9795403001143
472163    9990385651922
472164    9990385651939
472165    9990385651946
Name: producto_id, Length: 472166, dtype: object

In [327]:
#dataset precio semana 2
precio_semana2 = pd.read_json(dataset2)

In [321]:
#se buscan valores nulos en dataframe 2
busca_nulos(precio_semana2)

precio         0
producto_id    0
sucursal_id    0
dtype: int64

In [328]:
#tranformar a numero cuando la serie tiene distintos tipos de datos
precio_semana2['precio']=pd.to_numeric(precio_semana2.precio)

In [329]:
#pasar a flotante el precio
precio_semana2['precio'].astype(float)

0          29.90
1          39.90
2         499.99
3         539.99
4         519.99
           ...  
397729    139.99
397730     34.99
397731    312.50
397732    312.50
397733    198.90
Name: precio, Length: 397734, dtype: float64

In [335]:
#dataset precio semana 3
#pd.read_fwf(dataset4)
precio_semana3 = pd.read_csv(dataset3, delimiter="|", encoding=detect_encoding(dataset3))


In [336]:
#se buscan valores nulos en dataframe 3
busca_nulos(precio_semana3)

precio         1960
producto_id       6
sucursal_id       6
dtype: int64

In [351]:
#guardo los valores nulos. Ya que no hay casi información es imposible dar con estos datos. 
# Se prosigue a eliminar los 6 registros
precio_semana3[precio_semana3['producto_id'].isna()]

,precio,producto_id,sucursal_id
375118,203.0,NaN,NaN
375157,NaN,NaN,NaN
375193,240.0,NaN,NaN
375278,NaN,NaN,NaN
375357,NaN,NaN,NaN
375397,NaN,NaN,NaN


In [120]:
#dataset precio semana 4
precios_xlsx = pd.read_excel('Datasets_relevamiento_precios/precios_semanas_20200419_20200426.xlsx', sheet_name=None)

In [244]:
# unir todas las hojas en un solo dataframe y limpiarlo desde allí
precio_semana4 = pd.concat(precios_xlsx, ignore_index=True)

In [245]:
precio_semana4

,precio,sucursal_id,producto_id
0,399.00,2-1-092,2288.0
1,299.00,2-1-206,2288.0
2,399.00,2-2-241,2288.0
3,49999.00,9-1-430,205870.0
4,53999.00,9-2-4,205870.0
...,...,...,...
937447,139.99,2001-01-25 00:00:00,9569753142128
937448,34.99,2001-01-25 00:00:00,9795403001143
937449,312.50,2003-01-05 00:00:00,9990385651922
937450,312.50,2003-01-05 00:00:00,9990385651939


In [247]:
#convierto el dato str para poder manipularlo mejor
precio_semana4['sucursal_id']=precio_semana4['sucursal_id'].astype(str)

In [ ]:
def norm_fecha (x):
    '''
    Esta funcion intenta normalizar los datos que vienen cargados como datetime en alguna columna
    toma cada argumento seguido de '-' y los retorna los valores en el órden correcto
    '''
    
    if ' 00:00:00' in x:
        x = x.replace(' 00:00:00','')
        val_1 = x.split('-')[0]
        val_2 = x.split('-')[1]
        val_3 = x.split('-')[2]
       
        x = val_3 + '-' + val_2 + '-' + val_1
    return x

In [248]:
precio_semana4['sucursal_id']=precio_semana4['sucursal_id'].apply(norm_fecha)

In [249]:
precio_semana4

,precio,sucursal_id,producto_id
0,399.00,2-1-092,2288.0
1,299.00,2-1-206,2288.0
2,399.00,2-2-241,2288.0
3,49999.00,9-1-430,205870.0
4,53999.00,9-2-4,205870.0
...,...,...,...
937447,139.99,25-01-2001,9569753142128
937448,34.99,25-01-2001,9795403001143
937449,312.50,05-01-2003,9990385651922
937450,312.50,05-01-2003,9990385651939


In [167]:
precio_semana4

,precio,sucursal_id,producto_id
0,399.00,2-1-092,2288.0
1,299.00,2-1-206,2288.0
2,399.00,2-2-241,2288.0
3,49999.00,9-1-430,205870.0
4,53999.00,9-2-4,205870.0
...,...,...,...
937447,139.99,2001-01-25 00:00:00,9569753142128
937448,34.99,2001-01-25 00:00:00,9795403001143
937449,312.50,2003-01-05 00:00:00,9990385651922
937450,312.50,2003-01-05 00:00:00,9990385651939


In [45]:
# valores faltantes - al no ser un valor significativo se lo deja para que el Data Analyst los trate de ser necesario según el criterio del negocio
precio_semana4.isna().sum()

precio         3543
sucursal_id       0
producto_id       0
dtype: int64

In [138]:
precio_semana4['producto_id'] = precio_semana4['producto_id'].fillna(0)

In [141]:
precio_semana4[precio_semana4['producto_id'].isna()]

,precio,sucursal_id,producto_id


In [111]:
#elimino los números y unidades del nombre ya que parece un error de carga 
import re
precio_semana4['producto_id'] = precio_semana4['producto_id'].astype(str).apply(lambda x: x.strip('10-1-'))


In [180]:
precio_semana4['producto_id']=precio_semana4['producto_id'].astype(str).str.replace(" " ,"")

In [ ]:
precio_semana4['producto_id']=precio_semana4['producto_id'].str.replace(" " ,"")

In [ ]:
precio_semana4['producto_id'] = precio_semana4['producto_id'].astype(str).apply(lambda x: x.strip('10-1-'))

In [ ]:
#precio_semana4['producto_id'].apply(lambda x: x.replace(' ',''))

In [309]:
#pasar columna precio a flotante
precio_semana4['producto_id']=precio_semana4['producto_id'].astype(float)

ValueError: could not convert string to float: '10-1-2300006000003'

In [181]:
#dataset producto
producto = pd.read_parquet(dataset5, engine='pyarrow')

In [117]:
#tamaño de la tabla producto.shape (72038, 7)
#la tabla no tiene productos duplicados --> producto.drop_duplicates(inplace=True)
#shape despues de borrar duplicados es la misma #tamaño de la tabla (72038, 7)

In [170]:
#tamaño de la tabla
producto.shape

(72038, 7)

In [118]:
#tamaño de la tabla
producto.shape

(72038, 7)

In [107]:
#ver valores nulos 
producto.isnull().sum() #mismo resultado si uso .isna 

id                  0
marca               2
nombre              2
presentacion        2
categoria1      72034
categoria2      72034
categoria3      72034
dtype: int64

In [ ]:
#las columnas últimas 3 columnas tienen un % muy bajo de valores y es muy difícil inferir cuales serían los datos faltantes sólo en base a la información que tenemos.
#Elimino las columnas y si fuera un caso real, le muestro los siguientes porcentajes al cliente para su consideración. 
# Además le preguntaría si esa info es relevante y de serlo, propondría armar una tabla de categorías (id producto y categoría)

In [139]:
#guardo el total en una variable
total = producto.shape[0]


In [175]:
producto.categoria1.isnull().sum()

72034

In [176]:
nulos_cat1 = round((72034*100/total), 2)
nulos_cat2 = round((72034*100/total), 2)
nulos_cat3 = round((72034*100/total), 2)
print("total de faltantes categoría 1:", nulos_cat1, '%')
print("total de faltantes categoría 2:", nulos_cat3, '%')
print("total de faltantes categoría 3:", nulos_cat2, '%')

total de faltantes categoría 1: 99.99 %
total de faltantes categoría 2: 99.99 %
total de faltantes categoría 3: 99.99 %


In [182]:
#dropeo las ultimas columnas
producto.drop(['categoria1', 'categoria2', 'categoria3'], axis=1, inplace=True)

In [114]:
#veo cuales son los porductos sin 'nombre'
producto[producto['nombre'].isnull()]
#coinciden con los 2 null de marca

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
53619,7798024299045,None,None,None,Almacén,Aceites,Aerosol
55798,7798061190213,None,None,None,Almacén,Aceites,Oliva


In [183]:
#seteo el valor de datos faltantes en "sin dato"
producto['nombre'].fillna("Sin Dato", inplace = True)
producto['marca'].fillna("Sin Dato", inplace = True)
producto['presentacion'].fillna("Sin Dato", inplace = True)

In [38]:
producto['id'].value_counts().sum #no hay valores repetidos en en id :) se puede usar como clave primario de ser necesario

NameError: name 'producto' is not defined

In [190]:
#elimino los números y unidades del nombre ya que parece un error de carga 
producto.nombre = producto.nombre.astype(str).apply(lambda x: x.strip('0123456789UnKgGr '))

In [191]:
producto

,id,marca,nombre,presentacion
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima,1.0 un
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima,1.0 un
2,0000000205870,SIN MARCA,Ojo de Bife,1.0 kg
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito,1.0 kg
4,0000000205955,SIN MARCA,Chiquizuela Novillito,1.0 kg
...,...,...,...,...
72033,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Sitas,500.0 gr
72034,9795403001143,MAYO,Mini Pizzetas Mayo,12.0 un
72035,9990385651922,DANA,Te Negro en Hebras Lata Dana,50.0 gr
72036,9990385651939,DANA,Te Verde en Hebras Lata Dana,50.0 gr


In [7]:
#dataset sucursal
sucursal = pd.read_csv(dataset6, delimiter=",", encoding=detect_encoding(dataset6))

In [8]:
sucursal.head()

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado


In [10]:
# no hay valores nulos
sucursal.isnull().sum()

id                     0
comercioId             0
banderaId              0
banderaDescripcion     0
comercioRazonSocial    0
provincia              0
localidad              0
direccion              0
lat                    0
lng                    0
sucursalNombre         0
sucursalTipo           0
dtype: int64

In [13]:
#la tabla de Provincia contiene un Código que a simple vista parece coincidir con cada Provincia. 
#no se agrega info

In [14]:
#cambio nombre de las columnas lat y lng para que sea más claro
sucursal.rename(columns={'lat': 'Latitud', 
                            'lng': 'Longitud'}, inplace=True)


In [15]:
sucursal

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,Latitud,Longitud,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado
...,...,...,...,...,...,...,...,...,...,...,...,...
2328,9-3-5277,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-U,COMODORO RIVADAVIA,Yrigoyen Hipolito 0,-45.873300,-67.493500,Jumbo-Comodoro,Hipermercado
2329,9-3-5626,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-B,GENERAL PACHECO,Boulogne Sur Mer 1275,-34.474500,-58.625700,Jumbo Pacheco Novo,Hipermercado
2330,9-3-5961,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-C,CIUDAD AUTONOMA BUENOS AIRES,Avenida Santa Fe 4950,-34.577200,-58.430000,Jumbo Av. Santa Fé,Supermercado
2331,9-3-628,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-B,SAN FERNANDO,Avenida Del Libertador Gral San Martin 2271,-34.446900,-58.545700,Jumbo San Fernando,Supermercado


In [ ]:
#importar mysql.connector como conexión 
#importar pandas como pd
Try : 
    mydb = connection.connect(host="localhost", base de datos = 'Estudiante',usuario="raíz", passwd="raíz",use_pure=True) 
    consulta = "Seleccione * de los detalles del estudiante;" 
    result_dataFrame = pd.read_sql(query,mydb) 
    mydb.close() #cerrar la conexión
except Excepción como e: 
    mydb.close() 
    print(str(e))